<a href="https://colab.research.google.com/github/JorgeSedek/tp2_orga_datos/blob/main/TP2_S30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.n

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
rdd_movies = sqlContext.read.parquet('/content/drive/MyDrive/orgaDatos/Movie dataset/Parquets/movies.parquet').rdd
rdd_cast = sqlContext.read.parquet('/content/drive/MyDrive/orgaDatos/Movie dataset/Parquets/cast.parquet').rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
rdd_movies.take(1)

[Row(adult='False', belongs_to_collection='Toy Story Collection', budget='30000000', genres='Animation,Comedy,Family', homepage='http://toystory.disney.com/toy-story', id='862', imdb_id='tt0114709', original_language='en', original_title='Toy Story', overview="Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.", popularity=21.946943, poster_path='/rhIRbceoE9lR4veEXuwCC2wARtG.jpg', production_companies='3', production_countries="[{'iso_3166_1': 'US', 'name': 'United States of America'}]", release_date='1995-10-30', revenue=373554033.0, runtime=81.0, spoken_languages="[{'iso_639_1': 'en', 'name': 'English'}]", status='Released', tagline=None, title='Toy Story', video=False, vote_average=7.7, vote_count=5415.0)]

filtro los datos que tienen: \\
1. nans en id y budget.
2. budget igual a 0.
3. id en forma de fecha.


In [ ]:
rdd_pelicula_budget = rdd_movies.filter(lambda x: not pd.isnull(x.id) and (not "-" in x.id)  and (not pd.isnull(x.budget) and not (x.budget == '0'))).map(lambda x: (int(x.id), int(x.budget)))
rdd_pelicula_budget.take(5)

[(862, 30000000),
 (8844, 65000000),
 (31357, 16000000),
 (949, 60000000),
 (11860, 58000000)]

In [ ]:
rdd_movies.filter(lambda x:("-" in x.id)).take(1)

[Row(adult=' - Written by Ørnås', belongs_to_collection='0.065736', budget='/ff9qCepilowshEtG2GYWwzt2bs4.jpg', genres=None, homepage="[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'LU', 'name': 'Luxembourg'}, {'iso_3166_1': 'GB', 'name': 'United Kingdom'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]", id='1997-08-20', imdb_id='0', original_language='104.0', original_title="[{'iso_639_1': 'en', 'name': 'English'}]", overview='Released', popularity=None, poster_path='Midnight Man', production_companies=None, production_countries='6.0', release_date='1', revenue=None, runtime=None, spoken_languages=None, status=None, tagline=None, title=None, video=None, vote_average=None, vote_count=None)]

In [ ]:
rdd_cast.take(10)

[Row(id=862, cast_id=31, character='Woody (voice)', name='Tom Hanks'),
 Row(id=862, cast_id=12898, character='Buzz Lightyear (voice)', name='Tim Allen'),
 Row(id=862, cast_id=7167, character='Mr. Potato Head (voice)', name='Don Rickles'),
 Row(id=862, cast_id=12899, character='Slinky Dog (voice)', name='Jim Varney'),
 Row(id=862, cast_id=12900, character='Rex (voice)', name='Wallace Shawn'),
 Row(id=862, cast_id=7907, character='Hamm (voice)', name='John Ratzenberger'),
 Row(id=862, cast_id=8873, character='Bo Peep (voice)', name='Annie Potts'),
 Row(id=862, cast_id=1116442, character='Andy (voice)', name='John Morris'),
 Row(id=862, cast_id=12901, character='Sid (voice)', name='Erik von Detten'),
 Row(id=862, cast_id=12133, character='Mrs. Davis (voice)', name='Laurie Metcalf')]

S30 (). Estimamos el sueldo de un actor como la inversión dividida entre la cantidad de actores. En base a esta estimación, ¿cuál es el actor con el patrimonio más grande?

Tengo que: \\
- contar cuantos actores hay por id de pelicula. \\
- dividir el budget por la cantidad de actores. \\
- Después tengo que sumar ese valor por cada actor.

In [ ]:
rdd_cast.count()

562474

In [ ]:
rdd_cast.take(5)
rdd_cast_sin_na = rdd_cast.filter(lambda x: not pd.isnull(x.id) and (not pd.isnull(x.name)))

In [ ]:
pelicula_actor = rdd_cast_sin_na.map(lambda x: (x.id, x.name, 1)).distinct()
pelicula_actor.take(5)

[(862, 'Tim Allen', 1),
 (862, 'Jim Varney', 1),
 (862, 'Wallace Shawn', 1),
 (862, 'John Ratzenberger', 1),
 (862, 'Laurie Metcalf', 1)]

In [ ]:
cantidad_actores = pelicula_actor.map(lambda x: (x[0], x[2])).reduceByKey(lambda a,b: a + b)
cantidad_actores.take(5)

[(862, 13), (8844, 26), (15602, 7), (11862, 12), (11860, 57)]

In [ ]:
budget_actores = rdd_pelicula_budget.join(cantidad_actores)

In [ ]:
rdd_pelicula_budget.take(1000)

In [ ]:
cantidad_actores.first()

(862, 13)

In [ ]:
budget_actores.take(5)

[(8844, (65000000, 26)),
 (11860, (58000000, 57)),
 (1408, (98000000, 31)),
 (524, (52000000, 45)),
 (4584, (16500000, 17))]

In [ ]:
sueldo_pelicula = budget_actores.map(lambda x: (x[0],x[1][0]/x[1][1]))
sueldo_pelicula.take(5)

[(8844, 2500000.0),
 (11860, 1017543.8596491228),
 (1408, 3161290.3225806453),
 (524, 1155555.5555555555),
 (4584, 970588.2352941176)]

In [ ]:
sueldo_actor = sueldo_pelicula.join(pelicula_actor).map(lambda x: (x[1][1], x[1][0]))
sueldo_actor.take(5)

[('Robin Williams', 2500000.0),
 ('Jonathan Hyde', 2500000.0),
 ('Bebe Neuwirth', 2500000.0),
 ('David Alan Grier', 2500000.0),
 ('Adam Hann-Byrd', 2500000.0)]

In [ ]:
sueldo_actor.reduceByKey(lambda a, b: a + b).takeOrdered(5, key=lambda x: -x[1])

[('Samuel L. Jackson', 185869193.80069774),
 ('Nicolas Cage', 165677863.7127913),
 ('Bruce Willis', 156451313.26899275),
 ('Robert De Niro', 138347297.90670684),
 ('Eddie Murphy', 136250264.16661215)]

El actor con el patrimonio más grande es Samuel L. Jackson.